# refering TF serving url https://www.tensorflow.org/tfx/tutorials/serving/rest_simple

In [1]:
import numpy as np
import argparse
import os
import pickle
from PIL import Image
import tensorflow as tf

import numpy as np

In [2]:
# define the path to the training and testing directories
dataset_wave='./data/spiral/'
trainingPath = dataset_wave+ "training/"
testingPath = dataset_wave+"testing/"


def get_data(data_path):
    X=[]
    Y=[]
    # loop through each image path and fetch the features, then write to index per row 
    #for ext in ('*.gif', '*.png', '*.jpg','*.jpeg'):
    label_paths=['healthy','parkinson']
    for label_path in label_paths:
        img_dir=data_path+label_path
        img_files=os.listdir(img_dir)
        if label_path=='healthy':
            ls=[ Y.append(0) for i in range(len(img_files))]
        else:
            ls=[ Y.append(1) for i in range(len(img_files))]
        for im_name in img_files:
            img = Image.open(img_dir+'/'+im_name)
            img = img.convert('RGB')
            img = img.resize((224, 224), Image.BILINEAR)
            img = np.array(img)

            img=img/255
            X.append(img)
                
    Y=np.array(Y)
    X=np.array(X)
    return X, Y


Xtrain, Ytrain=get_data(trainingPath)
Xtest,Ytest=get_data(testingPath )

Xtrain.shape, Ytrain.shape, Xtest.shape, Ytest.shape

((72, 224, 224, 3), (72,), (42, 224, 224, 3), (42,))

## prepare a feed-forward neural network focus on recall as metric

In [3]:
# cosntruct custom metrics for monitoring 

import tensorflow.keras.backend as K
def f1_score(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2

    # How many relevant items are selected?
    recall = c1 / c3

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score


def precision(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2

    return precision


def recall(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    recall = c1 / c3

    return recall

## cosntruct the model and give input/output tensor names

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, Conv2DTranspose, MaxPooling2D, Dropout, Dense, Input, Flatten
import tensorflow.keras.backend as K
K.set_image_data_format('channels_last')
input_tensor = {
    "hand_drawing": Input(name="hand_drawing", shape=(224,224,3), dtype="float32")}
print(type(input_tensor['hand_drawing']))
def get_model(input_tensor):
    inputs = input_tensor['hand_drawing']
    # Step 1 - Convolution
    x = Conv2D(32, (3, 3), input_shape = (224, 224,3), activation = 'relu', padding="same")(inputs)

    # Step 2 - Pooling
    x = MaxPooling2D(pool_size = (2, 2))(x)

    # Adding a second convolutional layer
    x = Conv2D(32, (3, 3), activation = 'relu')(x)
    x = MaxPooling2D(pool_size = (2, 2))(x)

    # Adding a second convolutional layer
    x = Conv2D(32, (3, 3), activation = 'relu')(x)
    x = MaxPooling2D(pool_size = (2, 2))(x)


    # Adding a second convolutional layer
    x = Conv2D(32, (3, 3), activation = 'relu')(x)
    x = MaxPooling2D(pool_size = (2, 2))(x)

    # Step 3 - Flattening
    x = Flatten()(x)

    # Step 4 - Full connection
    x = Dense(units = 1024, activation = 'relu')(x)
    outputs = Dense(units = 1, activation = 'sigmoid', name='outputs',dtype="float32")(x)


    model = Model(inputs=[inputs], outputs=[outputs])
    return model
m=get_model(input_tensor)
m.summary()

<class 'tensorflow.python.framework.ops.Tensor'>
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hand_drawing (InputLayer)    [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 110, 110, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 55, 55, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 53, 53, 32)        9248      
_________________________________________________________________
max_pooling2

## compile and train the model

In [5]:

m.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [11]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
epochs=35
early_stop = EarlyStopping(monitor='val_accuracy', 
                           min_delta=0.001, 
                           patience=20, 
                           mode='max', 
                           verbose=1)

checkpoint = ModelCheckpoint('FC_spiral_parkinson.h5', 
                             monitor='val_accuracy', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='max', 
                             period=1)

m.fit(Xtrain,Ytrain, batch_size=24,validation_data=(Xtest,Ytest),
                    callbacks=[early_stop,checkpoint], epochs=epochs, shuffle=True)


Epoch 1/35
3/3 [==============================] - ETA: 0s - loss: 0.6530 - accuracy: 0.5972
Epoch 00001: val_accuracy improved from -inf to 0.66667, saving model to FC_spiral_parkinson.h5
3/3 [==============================] - 1s 272ms/step - loss: 0.6530 - accuracy: 0.5972 - val_loss: 0.6524 - val_accuracy: 0.6667
Epoch 2/35
3/3 [==============================] - ETA: 0s - loss: 0.6489 - accuracy: 0.6528
Epoch 00002: val_accuracy improved from 0.66667 to 0.78571, saving model to FC_spiral_parkinson.h5
3/3 [==============================] - 1s 256ms/step - loss: 0.6489 - accuracy: 0.6528 - val_loss: 0.6164 - val_accuracy: 0.7857
Epoch 3/35
3/3 [==============================] - ETA: 0s - loss: 0.6127 - accuracy: 0.6667
Epoch 00003: val_accuracy improved from 0.78571 to 0.83333, saving model to FC_spiral_parkinson.h5
3/3 [==============================] - 1s 285ms/step - loss: 0.6127 - accuracy: 0.6667 - val_loss: 0.5847 - val_accuracy: 0.8333
Epoch 4/35
3/3 [===========================

In [12]:
m.save('model.h5')

In [13]:
from tensorflow.keras.models import load_model
loaded_model=load_model('model.h5')

In [14]:
output=loaded_model.evaluate(Xtest,Ytest)
type(output)
print("accuracy on the testing dataset is :", output[1])

2/2 [==============================] - 0s 13ms/step - loss: 0.3778 - accuracy: 0.7857
accuracy on the testing dataset is : 0.7857142686843872


In [15]:
## save model 
tf.saved_model.save(m, "model_ckpt/")

INFO:tensorflow:Assets written to: model_ckpt/assets
